## Exemplo de uso Python OpenAI para versão 1.X

In [1]:
import openai
openai.__version__

'1.1.2'

In [2]:
import os
from openai import AzureOpenAI
from configparser import ConfigParser, ExtendedInterpolation
import httpx
import numpy as np

config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('config-v1.x.ini', 'UTF-8')

http_client = httpx.Client(verify='petrobras-ca-root.pem')

client = AzureOpenAI(
    api_key=config['OPENAI']['OPENAI_API_KEY'],  
    api_version=config['OPENAI']['OPENAI_API_VERSION'],
    # azure_endpoint=config['OPENAI']['OPENAI_API_BASE'],
    base_url=config['OPENAI']['AZURE_OPENAI_BASE_URL'],
    http_client=http_client
)

In [3]:
def get_embedding(text, engine, **kwargs):
    response = client.embeddings.create(
        # Contact your team admin to get the name of your engine or model deployment.  
        # This is the name that they used when they created the model deployment.
        input=text,
        model=engine,
        **kwargs
    )

    embeddings = response.data[0].embedding
    return np.array(embeddings)

# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, engine, max_response_tokens=500):
    
    response = client.chat.completions.create(
        model=engine,
        messages=messages,
        temperature=0.5,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

### Exemplo Chat API

In [4]:
base_system_message = "Você é um assistente que conhece tudo sobre podcasts. Você é capaz de responder perguntas sobre o podcast, sobre os episódios e sobre os convidados."
system_message = f"{base_system_message.strip()}"

user_message = "A área de tecnologia da Petrobras, chamada de TIC está criando um programa de podcast através do qual pretende levar aos colaboradores da empresa temas sobre tecnologia tais como ChatGPT e IA generativa, por exemplo. Alguma sugestão de nome para esse podcast? Indique por favor uma lista de dez nomes bacanas para esse podcast."

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, engine=config['OPENAI']['CHATGPT_MODEL'], max_response_tokens=max_response_tokens)
messages.append({"role": "assistant", "content": response})

print_conversation(messages)

[SYSTEM]
Você é um assistente que conhece tudo sobre podcasts. Você é capaz de responder perguntas sobre o podcast, sobre os episódios e sobre os convidados.

[USER]
A área de tecnologia da Petrobras, chamada de TIC está criando um programa de podcast através do qual pretende levar aos colaboradores da empresa temas sobre tecnologia tais como ChatGPT e IA generativa, por exemplo. Alguma sugestão de nome para esse podcast? Indique por favor uma lista de dez nomes bacanas para esse podcast.

[ASSISTANT]
1. "TechPetro: Inovação e Futuro"
2. "PetroTech: Ondas de Inovação"
3. "TIC Talks: Conversas sobre Tecnologia"
4. "Petro IA: A Tecnologia em Movimento"
5. "Onda Digital Petrobras"
6. "Conexão TIC: O Futuro Hoje"
7. "Petrocast: IA e Além"
8. "TIC Insights: Mergulhando na Tecnologia"
9. "PetroDigital: O Som da Inovação"
10. "TechFlow Petrobras: Onde a Tecnologia Flui"



### Exemplo Embeddings API

In [5]:
get_embedding(text="Olá, mundo!", engine=config['OPENAI']['EMBEDDINGS_MODEL'])

array([ 0.00364376, -0.00354613,  0.00319183, ..., -0.0101534 ,
        0.00180456, -0.02301522])

### Exemplo Completion API (deprecated)

A API de Completion foi descontinuada pela Open AI.